# Generische Programmierung mit Generics

#### Marcel Lüthi <br/> Departement Mathematik und Informatik

# Motivation

* Bisherige Lösung um Klassen für mehrere Typen verwendbar zu machen:
    * Nutzung von Supertyp Objekt

In [1]:
class Stack { 
    
    private Object[] data = new Object[1000]; // Vereinfachung: Feste grösse
    private int nElements = 0; // Anzahl Elemente im Stack
 
    public void push(Object element) {
        data[nElements] = element;     
        nElements += 1;
    }
    
    public Object pop() { 
        if (nElements <= 0) {
            return null;
        }
        else {
            nElements -= 1;
            return data[nElements];
       }
    }
    public int size() { return this.nElements; }
}

# Probleme

* Wir brauchen explizite downcasts
* Intention nicht klar
    * Speichern wir Objekte oder Listen oder strings?
* Nicht typesafe
    * Fehler zur Laufzeit - Keine Hilfe von Compiler
* Klassen müssen in Hierarchie gebracht werden
    * Nicht möglich ohne Source Code

In [29]:
Stack stack = new Stack();
stack.push("abc");
stack.push(5);

String s = (String) stack.pop();

EvalException: java.base/java.lang.Integer cannot be cast to java.base/java.lang.String

# Lösung: Java Generics

In [30]:
class Stack<E> { 
    
    private E[] data = (E[])new Object[1000];
    private int nElements = 0; // Anzahl Elemente im Stack
 
 
    public void push(E element) {
        data[nElements] = element;     
        nElements += 1;
    }
    
    public E pop() { 
        if (nElements <= 0) {
            return null;
        }
        else {
            nElements -= 1;
            return data[nElements];
       }
    }
    public int size() { return this.nElements; }
}

# Nutzung des Generischen Stacks

In [1]:
Stack<Integer> intStack = new Stack<Integer>();
intStack.push(new Integer(5));
Integer a = intStack.pop();


### Alternative Syntax

In [ ]:
Stack<Integer> intStack = new Stack<>();

#### Vorteile 

* Homogene Datenstruktur mit Compilezeit Typprüfung
* Typsicherheit (keine Casts)
* Intention viel klarer.

# Terminologie

<div style="width:60%">
<img src="images/generics-terminology.png"/>
</div>

# Mehrere Platzhalter

In [21]:
class Map<K, V> {

    K[] keys = (K[]) new Object[1000];
    V[] values = (V[]) new Object[1000];
    int nElements = 0;

    void put(K key, V value) {
        keys[nElements] = key;
        values[nElements] = value;
        nElements += 1;
    }

    V get(K key) {
        for (int i = 0; i < nElements; ++i) { 
            if (keys[i].equals(key)) { 
                return values[i];
            }
        }
        return null;
    }

}

# Anwendung von Map: Geburtstage speichern

In [27]:
Map<String, Date> map = new Map<>();
map.put("sonja", new Date(10, 02, 2000));
map.put("tom", new Date(10, 02, 1997));
map.put("leslie", new Date(7, 05, 1999));

Date tomsBirthday = map.get("tom");

# Type erasure

* Zur Kompilationszeit werden alle Typparameter ersetzt

![erasure](images./generics-erasure.png)

# Rohtypen

* Rohtypen können auch in Zuweisungen verwendet werden:

In [149]:
class Stack<E> {}
Stack s = new Stack<Integer>();

Achtung!
```Stack``` ist nicht dasselbe wie ```Stack<Object>```

Das funktioniert

In [150]:
Stack s = new Stack<Integer>();

aber das nicht:

In [152]:
Stack<Object> s = new Stack<Integer>();

CompilationException: 

> Rohtypen sind nur aus Ko

# Erzeugen von Objekten von Generischem Typ

> Nicht möglich!

In [34]:
class Foo<T> {
    T t = new T();
}

CompilationException: 


* Keine Typinformation zur Laufzeit verfügbar.
    * Wie viel Speicher soll reserviert werden?

# Erzeugen von generischen Arrays

* Folgendes funktioniert nicht:

In [35]:
class Foo<T> {
   T[] t = new T[100];
}

CompilationException: 

* Funktionierender Hack!

In [36]:
class Foo<T> {
   T[] arrayOfTs = (T[]) new Object[100];
}

# Gefahr

In [68]:
class Foo<T> {
    T[] arrayOfTs = (T[]) new Object[100];

    T bang() { 
        Object[] o = arrayOfTs;
        o[0] = "abc";     
        return arrayOfTs[0];
    }
}

* Was passiert hier?

In [69]:
Foo<Integer> bang = new Foo<>();
bang.bang();

EvalException: java.base/java.lang.String cannot be cast to java.base/java.lang.Integer

# Lösung: Factories

* Factory sind einfache Klassen um Objekte zu erzeugen.
* Saubere Alternative um Objekte von generischem Typ zu erzeugen
* Verbreitetes Muster in der Softwareentwicklung 

In [66]:
interface Factory<T> { 
    public T createInstance();
}


# Factory Implementation

In [67]:
class IntegerFactory implements Factory<Integer> { 
    int value = 0; 
    public IntegerFactory(int i)  {
        this.value = value;
    }
    public Integer createInstance() { 
        return new Integer(value);
    }
}

* Entsprechend auch für Strings, Dates, Integer[], ...

# Übung

* Schreiben Sie eine Factory die Arrays von Integern erzeugt.

# Nutzung der Factory

In [57]:
class Foo<T> {

    T t = null;
    
    public Foo(Factory<T> factory) { 
        t = factory.createInstance();
    }
}

In [63]:
Foo<Integer> f = new Foo<>(new IntegerFactory(0));

# Typeinschränkungen 

# Typeinschränkungen (Bounded type parameters)

* Wir können Typeinschränkung auf generischen Parameter definieren
* Beispiel: Typ sollte vergleichbar sein: ```class A<E extends Comparable>```
    * Nur Subtypen von Comparable können als generische Typen benutzt werden.     
* Mehrere Einschränkungen sind möglich
* Beispiel: Typ sollte vergleichbar und serialisierbar sein: ```class A<E extends Comparable & Serializable>```

# Beispiel

In [72]:
class SortedList <E extends Comparable> {
      E[] data = (E[]) new Comparable[1000];
      int nElements = 0;

      void add(E elem) {
          int i = nElements - 1;
          while (i >= 0 && elem.compareTo(data[i]) > 0) {  
              data[i+1] = data[i];
              i -= 1;  
          }
          data [i+1] = elem; 
          nElements++;          
     }
    
}

# Instanzierung 

Folgendes funktioniert:

In [96]:
new SortedList<Integer>();
new SortedList<String>();
new SortedList<Comparable>();

REPL.$JShell$96D$SortedList@5fde2c20

Folgendes funktioniert nicht:

In [98]:
new SortedList<Stack> ();
new SortedList<Object>();

CompilationException: 

# Vererbung

Einfach zu begreifen, wenn Sie folgendes verinnerlicht haben.

Für Klasse ```A<T>``` gilt:
* ```A``` bedeutet ```A<Object>```
    * Konkrete Klasse
* ```A<T>``` ist:
    * Konkrete Klasse für fixes T
    * Generische Klasse wenn T Typparameter ist

# Übung

In [92]:
class Foo<T> {}

Welche dieser Statements funktioniert? Weshalb?

In [88]:
class Bar extends Foo {}

In [89]:
class Bar<T> extends Foo<T> {}

In [106]:
class Baz extends Foo<T> {}

CompilationException: 

# Übung

In [96]:
class Foo {}
class Bar<T> extends Foo {}
class Baz<T, U> extends Foo {}

Welche dieser Statements funktioniert? Weshalb?

In [97]:
Foo f = new Bar<Integer>()

In [107]:
Foo g = new Baz<Integer, Float>()

CompilationException: 

# Übung

In [101]:
class Foo<T> {}
class Bar<T> extends Foo<T> {}
class Baz<T, U> extends Foo<T> {}

Welche der folgenden Statements funktioniert? Weshalb?

In [102]:
Foo<Integer> f = new Bar<Integer>();

In [103]:
Foo<Integer> g = new Baz<Integer, Float>();

In [105]:
Foo<Short> h = new Bar<Integer>();

CompilationException: 

# Überschreiben von Methoden

In [110]:
class Foo<T> { 
    void f(T t) {}
}

* Wenn von konkreter Methode geerbt:
    * ```T``` wird durch konkreten Typ ersetzt

In [114]:
class Bar extends Foo<Integer> { 
    void f(Integer i) {}
}

* Wenn von generischer Klasse geerbt
    * ```T``` bleibt Typparameter

In [116]:
class Bar<T> extends Foo<T> { 
    void f(T t) {}
}

# Wildcards

# Covariance, Contravariance und Invariance

## Covariance
Falls ```A``` ein Supertyp von ```B``` ist, dann ist ```Foo<A>``` ein Supertyp von ```Foo<B>```

## Contravariance
Falls ```A``` ein Supertyp von ```B``` ist, dann ist ```Foo<A>``` ein Subtyp von ```Foo<B>```

## Invariance
Falls ```A``` ein Supertyp von ```B``` ist, dann ist ```Foo<A>``` weder ein Subtyp von ```Foo<B>``` noch ein Supertyp

# Covariance, Contravariance und Invariance

Arrays sind Covariant in Java

In [123]:
Object[] objectArray = new String[100]


Alle anderen Generischen Typen sind Invariant

In [133]:
class List<T> {}

In [131]:
List<Object> o = new List<Integer>();

CompilationException: 

# Motivation

> Gemeinsamer Basistyp für Generische Objekte
>    * Unabhängig von Parameter

In [135]:
class List<T> {}

In [139]:
List<?> l = null;
l = new List<Integer>();
l = new List<String>();
l = new List<Object>();

# Kompatibilitätsbeziehungen

In [143]:
class A<T> {}
class B<T> extends A<T> {}

In [146]:
A<?> a = new A<String>();
A<?> a = new B<String>();
A a = new A<String>();
Object a = new A<String>();


CompilationException: 